In [ ]:
import pandas as pd
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, MultiLineString

In [ ]:
# read DC polygon
gdf_temp = gpd.read_file("../../data/dc/DC_Quadrants-shp/DC_Quadrants.shp")
se_poly = gdf_temp[gdf_temp['QUADRANT'] == 'SE'].geometry.loc[1]

print("Here's OSMNX on walking: ", ox.downloader._get_osm_filter("walk"))

cf = '["highway"~"trunk|trunk_link|tertiary|tertiary_link|secondary|residential|primary|primary_link"]'
G_roads_init = ox.graph.graph_from_polygon(se_poly, network_type="walk", custom_filter=cf, retain_all = True, )


In [ ]:
nx.set_edge_attributes(G_roads_init, 0, 'oneway')
ox.io.save_graph_shapefile(G_roads_init, "../../data/dc/dc_roads_init.shp")

# Can load from here

In [23]:
import pandas as pd
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, MultiLineString, Point


In [24]:

dc_roads_init_gdf = gpd.read_file("../../data/dc/dc_roads_init.shp/edges.shp")
dc_roads_init_gdf.drop(inplace=True, columns=['highway', 'junction','bridge','access','maxspeed','oneway','ref','tunnel'])
dc_roads_init_gdf.crs == "EPSG:4326"

True

Not totally sure what to/from is about but let's drop it

In [25]:
dc_roads_init_gdf.drop(inplace=True, columns=['to','from'])

Sick:

`[road for road in list(dc_roads_init_gdf.geometry) if type(road) != LineString] `

This makes life much easier than in Brighton!

In [26]:
OFFSET = 1
dc_roads_init_gdf['sw_left'] = OFFSET
dc_roads_init_gdf['sw_right'] = OFFSET
dc_roads_init_gdf['id'] = dc_roads_init_gdf.osmid
crs_local = "EPSG:26785"
crs_global = "EPSG:4326"

In [27]:
dc_roads_init_gdf = dc_roads_init_gdf.to_crs(crs_local)

In [28]:
import sidewalkify as swky
sidewalk_paths = swky.graph.graph_workflow(dc_roads_init_gdf)
dc_sidewalks = swky.draw.draw_sidewalks(sidewalk_paths)

oops


/home/adam/anaconda3/envs/wheelway/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [29]:
def safe_int(maybe_int):
    if type(eval(maybe_int)) == list:
        return int(eval(maybe_int)[0])
    else:
        return int(maybe_int)

dc_sidewalks['street_id'] = dc_sidewalks['street_id'].map(safe_int)

In [30]:
dc_sidewalks.to_file("../../data/dc/dc_sidewalks")

In [31]:
def multis_to_lines(ls_or_mls):
    if type(ls_or_mls) == LineString:
        return [ls_or_mls]
    else:
        return list(ls_or_mls)

def explode_series(ser):
    return ser.apply(pd.Series).stack().reset_index(drop=True)
    
#geoseries = pd.Series(dc_sidewalks.geometry).map(lambda x : x.coords)
points_arr = explode_series(explode_series(dc_sidewalks.geometry.map(multis_to_lines)).map(lambda x : x.coords)).unique()
points = list(points_arr)

In [32]:
xs = list(map(lambda x : x[0], points))
ys = list(map(lambda x : x[1], points))
df = pd.DataFrame({'y':ys, 'x':xs, 'id':range(len(xs))})
gdf_new_pts = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(xs,ys))

gdf_new_pts.crs = crs_local

In [33]:
import shapely.ops as ops

def snap_endpoints(linestring, geom, tol):
    linelist = linestring.coords
    assert len(linelist) == 2
    p0 = ops.snap(Point(linelist[0]), geom, tol)
    p1 = ops.snap(Point(linelist[-1]), geom, tol)  
    #return LineString([Point(p0)] + list(map(Point, linelist[1:-1])) + [Point(pn)])
    return LineString([Point(p0), Point(p1)])

def round_pt(pt):
    a, b = pt
    a = round(a,1)
    b = round(b,1)
    return Point(a,b)

def round_edge(edge):
    try:
        a, b = tuple(edge.coords)
    except:
        print(edge)
        raise
    a = round_pt(a)
    b = round_pt(b)
    return LineString([a,b])

In [34]:
dc_sidewalks = dc_sidewalks.drop(3890)

In [35]:
def ls_to_mls(linestring):
    if type(linestring) == MultiLineString:
        return linestring
    else:
        linelist = linestring.coords
        pairlist = list(zip(linelist, linelist[1:]))
        return MultiLineString(pairlist)

dc_sidewalks['geometry'] = dc_sidewalks.geometry.map(ls_to_mls)
dc_sidewalks = dc_sidewalks.explode().reset_index(drop=True)

In [36]:
dc_sidewalks[dc_sidewalks.geometry.map(lambda x : len(x.coords)) != 2]

,forward,street_id,geometry


In [39]:
#all_sidewalks = ops.unary_union(pd.Series(dc_sidewalks.geometry))
dc_sidewalks.crs = crs_local

In [46]:
dc_sidewalks.crs
dc_sidewalks = dc_sidewalks.to_crs(crs_global)

In [48]:
dc_sidewalks

,forward,street_id,geometry
0,1,6059999,LINESTRING (-76.93237632992584 38.888319646618...
1,0,6059999,LINESTRING (-76.93222733409196 38.888994629425...
2,0,6059999,LINESTRING (-76.93221033414997 38.889074629397...
3,0,6059999,LINESTRING (-76.93221033093192 38.889074644287...
4,1,590581519,LINESTRING (-76.93207233738202 38.889702631245...
5,0,397252651,LINESTRING (-76.93278516900581 38.889702234085...
6,0,397252651,LINESTRING (-76.93301315426447 38.889702434238...
7,0,397252651,LINESTRING (-76.93328263625834 38.889700534562...
8,0,397252651,LINESTRING (-76.933282684693 38.88970053448227...
9,1,136558395,LINESTRING (-76.93416920207598 38.889703832600...


In [49]:
dc_sidewalks.to_file("../../data/dc/dc_sidewalks_processed.shp")

In [43]:
dc_sidewalks = gpd.read_file("../../data/dc/dc_sidewalks_processed.shp")

## break for crossify

In [50]:
def custom_crosswalks(streetmap):
    pass

# at each intersection:
# determine the number of the intersection
# if > 2, search for the three closest nodes
# if they aren't reachable from each other, add a crosswalk

,layer,highway,footway,geometry
0,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
1,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
2,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
3,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
4,0,footway,crossing,"LINESTRING (-77 38.8, -77 38.8)"
5,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
6,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
7,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
8,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"
9,0,footway,crossing,"LINESTRING (-77 38.9, -77 38.9)"


In [ ]:
len(dc_crosswalks.geometry.unique())